In [1]:
import mlflow

In [2]:
mlflow.set_tracking_uri('sqlite:///mlflow.db')
mlflow.set_experiment('nyc-taxi-experiment')

<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment_tracking/mlruns/1', creation_time=1747436587009, experiment_id='1', last_update_time=1747436587009, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
import seaborn as sns
import matplotlib as plt
from sklearn.metrics import mean_squared_error
import pickle

In [4]:
df = pd.read_parquet("data/yellow_tripdata_2023-01.parquet")
val_df = pd.read_parquet('data/yellow_tripdata_2023-02.parquet')

In [5]:
df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60
val_df['duration'] = (val_df['tpep_dropoff_datetime'] - val_df['tpep_pickup_datetime']).dt.total_seconds() / 60

df = df.query('duration >= 1 and duration <= 60')
val_df = val_df.query('duration >= 1 and duration <= 60')

In [6]:
categorical = ['PULocationID', 'DOLocationID']
df[categorical] = df[categorical].astype(str)
val_df[categorical] = val_df[categorical].astype(str)

In [7]:
train_dict = df[categorical].to_dict(orient='records')
val_dict = val_df[categorical].to_dict(orient='records')

In [8]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dict)
X_val = dv.transform(val_dict)

target = 'duration'
y_train = df[target].values
y_val = val_df[target].values

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

In [ ]:
with open('models/lin_reg.pkl', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [ ]:
y_pred = lr.predict(X_val)

In [ ]:
float(np.sqrt(mean_squared_error(y_val, y_pred)))

In [ ]:
with mlflow.start_run():
    mlflow.set_tag('developer', 'tman0004')

    mlflow.log_param('train-data-path', 'data/yellow_tripdata_2023-01.parquet')
    mlflow.log_param('valid-data-path', 'data/yellow_tripdata_2023-02.parquet')

    alpha = 0.01
    mlflow.log_param('alpha', alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = float(np.sqrt(mean_squared_error(y_val, y_pred)))
    mlflow.log_metric('rmse', rmse)

In [9]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [ ]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

: 

In [ ]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = float(np.sqrt(mean_squared_error(y_val, y_pred)))
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

In [ ]:
import mlflow.xgboost

params = {
    'learning_rate': 0.1233407742765768,
    'max_depth': 30,
    'min_child_weight': 13.092247284357976,
    'objective': 'reg:linear',
    'reg_alpha': 0.3399656070101837,
    'reg_lambda': 0.35806973799616537,
    'seed': 42
}

mlflow.xgboost.autolog()

booster = xgb.train(
    params=params,
    dtrain=train,
    num_boost_round=300,
    evals=[(valid, 'validation')],
    early_stopping_rounds=50
)

2025/05/17 00:35:32 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '1f0076e03c364368afdb12233fc621df', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
